In [ ]:
import os, platform, subprocess, sys, requests, json
import pandas as pd
from pandas import json_normalize
import matplotlib.pyplot as plt
import numpy as np
from scipy import signal
from IPython.display import Audio

In [ ]:
"""
Finding and importing provided libraries regardless of system and platform type
"""
## find and import libraries relative to this file's location (regardless of script or notebook)
## platfrom specific naming already handled within the lib directory's __init__ file
if '__file__' not in globals():
    sys.path.append(os.path.dirname(os.path.abspath('').split('transactions')[0]))
else:
    sys.path.append(os.path.dirname(__file__).split('transactions')[0])

from lib.encoder  import encode_tx, encode_script
from lib.helper   import decode_address
from lib.hash     import hash160, hash256, sha256
from lib.rpc      import RpcSocket
from lib.rpcauth  import rpcauth
from lib.sign     import sign_tx

In [ ]:
'''
Step 0.0: setup our RPC socket like we did in the previous example
        If you have not done the previous example, now would be the time.
        Please copy in the variables you used (located conveniently at the end of the previous example)
'''

nodeIP = "127.0.0.1"
nodePort = "18444"
username = "user"
password = "password"
wallet = "test2"

rpc = RpcSocket({ 'wallet': wallet,
                  'username': username,
                  'password': password,
                  'url' : nodeIP,
                  'port': nodePort})


In [ ]:
'''
Function to iterate backwards through 'N' number of latest blocks in the chain
'''
def getBlockHashList(count):
    if count < 1:
        return []
    chain_height = rpc.getBlockCount()
    if chain_height < count:
        max_block_count = chain_height
    else:
        max_block_count = count
    cur_block_count = 1
    cur_block_hash = rpc.getChain()['bestblockhash']
    block_hash_list = [cur_block_hash]

    while cur_block_count < max_block_count:
        prev_block_hash = rpc.getBlockHeader(cur_block_hash)['previousblockhash']
        cur_block_count += 1
        block_hash_list.append(prev_block_hash)
        cur_block_hash = prev_block_hash

    return block_hash_list

In [ ]:
'''
Get the hashes the top N blocks
'''
block_list = getBlockHashList(100)

In [ ]:
'''
Converting the block info into a pandas dataframe
'''
df = pd.DataFrame()
for block_hash in block_list:
    block = rpc.getBlock(block_hash)
    stats = rpc.getBlockStats(block_hash)
    block.update(stats)
    df = pd.concat([pd.DataFrame.from_dict(block, orient='index').T, df], axis=0, join='outer', ignore_index=True)

print(df.columns)
df

Fun with Music

In [ ]:
min_nTx = df["nTx"].min()
max_nTx = df["nTx"].max()
avg_nTx = round(df["nTx"].mean())
med_nTx = df["nTx"].median()
std_nTx = round(df["nTx"].std())
std2_nTx = round(df["nTx"].std())
print("Transaction statistics:\n  min:    " + str(min_nTx) + "\n  max:    " + str(max_nTx) + "\n  avg:    " + str(avg_nTx) + "\n  median: " + str(med_nTx) + "\n  std:    " + str(std_nTx))

In [ ]:
'''
Define note frequencies (using A = 440)
'''
A4  = 440.00
Bb4 = 466.16
B4  = 493.88
C5  = 523.25
Db5 = 554.37
D5  = 587.33
Eb5 = 622.25
E5  = 659.25
F5  = 698.46
Gb5 = 5739.99
G5  = 783.99
Ab5 = 830.61

In [ ]:
def note(freq, wave="sin", sec=1, framerate=10000):
    t = np.linspace(0, sec, framerate * sec) 
    if wave == "sin":
        return np.tan(2 * np.pi * freq * t)
    elif wave == "square":
        return signal.square(2 * np.pi * freq * t)
    elif wave == "triangle":
        return signal.sawtooth(2 * np.pi * freq * t)
    else:
        return np.cos(2 * np.pi * freq * t)
    
def play(data, framerate=10000):
    return Audio(data, rate=framerate, autoplay=True)

In [ ]:
# play three note [A4 to B4] for 1 second each
print(note(0, wave="sin").shape)
play(np.concatenate((note(1567.98), note(A4), note(Bb4, wave="triangle"), note(B4, wave="square")), axis=0))

In [ ]:
def getNotes(df, max_nTx, min_ntx, wave="sin"):
    _notes = note(0, wave)
    raw_range = max_nTx - min_ntx
    note_range = round(raw_range/12)
    for x in df["nTx"]:
        _note = C5
        note_type = round((x-min_ntx)/note_range)
        match note_type:
            case 0:
                _note = A4
            case 1:
                _note = Bb4
            case 2:
                _note = B4
            case 3:
                _note = C5
            case 4:
                _note = Db5
            case 5:
                _note = D5
            case 6:
                _note = Eb5
            case 7:
                _note = E5
            case 8:
                _note = F5
            case 9:
                _note = Gb5
            case 10:
                _note = G5
            case 11:
                _note = Ab5
            case _:
                _note = 0
        _note = note(_note, wave)
        _notes = np.concatenate((_notes, _note), axis=0)
        
    return _notes

In [ ]:
notes = getNotes(df, max_nTx, min_nTx, "square")

In [ ]:
play(notes)